In [1]:
import numpy as np
import einsteinpy as enp
import scipy as sp
import sympy as smp
import astropy.units as u
import einsteinpy as epy
from einsteinpy.coordinates import BoyerLindquistDifferential
from einsteinpy.metric import Kerr
from einsteinpy.symbolic import ChristoffelSymbols, MetricTensor

In [2]:
%load_ext snoop

In [3]:
import itertools


def levi_cevita_tensor(dim):
    arr = np.zeros(tuple([dim for _ in range(dim)]))
    for x in itertools.permutations(tuple(range(dim))):
        mat = np.zeros((dim, dim), dtype=np.int32)
        for i, j in zip(range(dim), x):
            mat[i, j] = 1
        arr[x] = int(np.linalg.det(mat))
    return arr

In [4]:
def antisymmetrize(arr):
    arr = np.swapaxes(arr, -2, -1)
    anti_arr = arr - np.swapaxes(arr, -2, -1)
    anti_tensor=np.swapaxes(anti_arr,-2,-1)
    return anti_tensor

In [5]:
def symmetrize(arr):
    #! Need To Work Over This
    toput = np.diagonal(arr, 0, 1, 2)
    de_ = np.zeros((4, 4, 4))


    idx = np.arange(de_.shape[0])
    de_[:, idx, idx] = toput

    return arr+np.transpose(arr,(0,2,1))-de_

In [6]:
def metric_tensor(a, r, theta,M=1):
    """
    Define the metric tensor function.

    Parameters:
        a (float): Parameter 'a' in the metric.
        r (float): Parameter 'r' in the metric.
        theta (float): Parameter 'theta' in the metric.

    Returns:
        np.ndarray: The metric tensor at the given values of a, r, and theta.
    """
    g = np.array(
        [
            [
                (-(a**2) + 2 * M * r - r**2 + a**2 * np.sin(theta) ** 2)
                / (r**2 + a**2 * np.cos(theta) ** 2),
                0,
                0,
                -(
                    (2 * a * M * r * np.sin(theta) ** 2)
                    / (r**2 + a**2 * np.cos(theta) ** 2)
                ),
            ],
            [
                0,
                (r**2 + a**2 * np.cos(theta) ** 2) / (a**2 - 2 * M * r + r**2),
                0,
                0,
            ],
            [0, 0, r**2 + a**2 * np.cos(theta) ** 2, 0],
            [
                -(
                    (2 * a * M * r * np.sin(theta) ** 2)
                    / (r**2 + a**2 * np.cos(theta) ** 2)
                ),
                0,
                0,
                (
                    np.sin(theta) ** 2
                    * (
                        (a**2 + r**2) ** 2
                        - a**2 * (a**2 - 2 * M * r + r**2) * np.sin(theta) ** 2
                    )
                )
                / (r**2 + a**2 * np.cos(theta) ** 2),
            ],
        ]
    )

    return g

In [7]:
def kerr_christoffel(r, theta, a, M=1):
    cs = np.zeros((4, 4, 4))

    # Definitions
    Delta = a**2 - 2 * M * r + r**2
    scA = (r**2 + a**2) ** 2 - Delta * a**2 * np.sin(theta) ** 2
    omega_k = 2 * M * a * r / scA
    Sigma = r**2 + a**2 * np.cos(theta) ** 2

    cs[3, 0, 1] = M * (2 * r**2 - Sigma) / (Delta * Sigma**2) * a
    cs[0, 0, 1] = cs[3, 0, 1] * (r**2 + a**2) / a

    cs[3, 0, 2] = -2 * M * a * r / (np.tan(theta) * Sigma**2)
    cs[0, 0, 2] = a * np.sin(theta) ** 2 * cs[3, 0, 2]

    cs[0, 1, 3] = (
        -M
        * a
        * (2 * r**2 * (r**2 + a**2) + Sigma * (r**2 - a**2))
        * np.sin(theta) ** 2
        / (Delta * Sigma**2)
    )

    cs[3, 1, 3] = (
        r * Sigma * (Sigma - 2 * M * r)
        - M * a**2 * (2 * r**2 - Sigma) * np.sin(theta) ** 2
    ) / (Delta * Sigma**2)

    cs[0, 2, 3] = M * a**3 * r * np.sin(theta) ** 2 * np.sin(2 * theta) / Sigma**2

    cs[3, 2, 3] = (scA - Sigma * a**2 * np.sin(theta) ** 2) / (
        np.tan(theta) * Sigma**2
    )

    cs[1, 0, 0] = M * Delta * (2 * r**2 - Sigma) / Sigma**3

    cs[1, 0, 3] = -cs[1, 0, 0] * a * np.sin(theta) ** 2

    cs[2, 0, 0] = -M * a * r * np.sin(2 * theta) / Sigma**3 * a
    cs[2, 0, 3] = -cs[2, 0, 0] * (r**2 + a**2) / a

    cs[1, 1, 1] = r / Sigma + (M - r) / Delta

    cs[1, 2, 2] = -r * Delta / Sigma
    cs[2, 1, 2] = -cs[1, 2, 2] / Delta

    cs[1, 1, 2] = cs[2, 2, 2] = -(a**2) * np.sin(2 * theta) / (2 * Sigma)
    cs[2, 1, 1] = -cs[1, 1, 2] / Delta

    cs[1, 3, 3] = (
        -Delta
        * (r * Sigma**2 - M * a**2 * (2 * r**2 - Sigma) * np.sin(theta) ** 2)
        * np.sin(theta) ** 2
        / Sigma**3
    )

    cs[2, 3, 3] = (
        -(Delta * Sigma**2 + 2 * M * r * (r**2 + a**2) ** 2)
        * np.sin(2 * theta)
        / (2 * Sigma**3)
    )

    return symmetrize(cs)
    # return cs

In [8]:
def kerr_riemann_tensor(a,r, theta, M=1,config='ulll'):
    # Define variables
    # a, r, theta = np.symbols('a r theta')

    # Components of the Riemann tensor for Kerr Metric
    rijkl = np.zeros((4, 4, 4, 4))

    X = r**2 - 3 * a**2 * np.cos(theta) ** 2
    Y = 3 * r**2 - a**2 * np.cos(theta) ** 2

    # Definitions
    Delta = a**2 - 2 * M * r + r**2
    scA = (r**2 + a**2) ** 2 - Delta * a**2 * np.sin(theta) ** 2
    omega_k = 2 * M * a * r / scA
    Sigma = r**2 + a**2 * np.cos(theta) ** 2

    rijkl[0, 0, 0, 3] = 2 * M**2 * a * r**2 * X * np.sin(theta) ** 2 / Sigma**4
    rijkl[3, 3, 0, 3] = -rijkl[0, 0, 0, 3]
    rijkl[0, 3, 0, 3] = -rijkl[0, 0, 0, 3] / omega_k
    rijkl[3, 0, 0, 3] = -rijkl[0, 0, 0, 3] / (
        2 * M * a * r / (Delta - a**2 * np.sin(theta) ** 2)
    )

    rijkl[0, 0, 1, 2] = -(
        M**2 * a**2 * r * Y * np.sin(2 * theta) / (Delta * Sigma**3)
    )
    rijkl[3, 3, 1, 2] = -rijkl[0, 0, 1, 2]
    rijkl[0, 3, 1, 2] = -rijkl[0, 0, 1, 2] / omega_k
    rijkl[3, 0, 1, 2] = -rijkl[0, 0, 1, 2] / (
        2 * M * a * r / (Delta - a**2 * np.sin(theta) ** 2)
    )

    rijkl[3, 2, 2, 3] = -(
        M * r * X * (2 * (r**2 + a**2) + a**2 * np.sin(theta) ** 2) / Sigma**3
    )

    rijkl[0, 1, 0, 1] = -rijkl[3, 2, 2, 3] / Delta

    rijkl[0, 2, 0, 2] = -(
        M * r * X * ((r**2 + a**2) + 2 * a**2 * np.sin(theta) ** 2) / Sigma**3
    )

    rijkl[3, 1, 1, 3] = -rijkl[0, 2, 0, 2] / Delta

    rijkl[0, 1, 0, 2] = rijkl[3, 2, 1, 3] = (
        -M
        * a**2
        / (Delta * Sigma**3)
        * Y
        * (3 * (r**2 + a**2) - 2 * M * r)
        * np.sin(theta)
        * np.cos(theta)
    )

    rijkl[0, 2, 0, 1] = rijkl[3, 2, 1, 3] = (
        -M
        * a**2
        / (Delta * Sigma**3)
        * Y
        * (3 * (r**2 + a**2) - 4 * M * r)
        * np.sin(theta)
        * np.cos(theta)
    )

    rijkl[3, 2, 0, 2] = -3 * M * a * r * X / Sigma**3
    rijkl[3, 1, 0, 1] = -rijkl[3, 2, 0, 2] / Delta

    rijkl[0, 2, 2, 3] = rijkl[3, 2, 0, 2] * np.sin(theta) ** 2 * (r**2 + a**2)
    rijkl[0, 1, 1, 3] = -rijkl[0, 2, 2, 3] / Delta

    rijkl[1, 0, 0, 2] = (
        -3 * M * a**2 * Delta / Sigma**4 * Y * np.sin(theta) * np.cos(theta)
    )
    rijkl[2, 0, 0, 1] = rijkl[1, 0, 0, 2] / Delta

    rijkl[1, 0, 1, 3] = (
        M
        * a
        * r
        / Sigma**4
        * X
        * np.sin(theta) ** 2
        * (3 * (r**2 + a**2) - 4 * M * r)
    )
    rijkl[1, 3, 0, 1] = -rijkl[1, 0, 1, 3]

    rijkl[2, 0, 2, 3] = -(
        M
        * a
        * r
        / Sigma**4
        * X
        * np.sin(theta) ** 2
        * (3 * (r**2 + a**2) - 2 * M * r)
    )

    rijkl[2, 3, 0, 2] = -rijkl[2, 0, 2, 3]

    rijkl[1, 0, 2, 3] = (
        -M
        * a
        * Delta
        / Sigma**4
        * Y
        * np.sin(theta)
        * np.cos(theta)
        * (2 * (r**2 + a**2) + a**2 * np.sin(theta) ** 2)
    )
    rijkl[2, 3, 0, 1] = -rijkl[1, 0, 2, 3] / Delta

    rijkl[1, 3, 0, 2] = (
        M
        * a
        * Delta
        / Sigma**4
        * Y
        * np.sin(theta)
        * np.cos(theta)
        * ((r**2 + a**2) + 2 * a**2 * np.sin(theta) ** 2)
    )

    rijkl[2, 0, 1, 3] = -rijkl[1, 3, 0, 2] / Delta

    rijkl[1, 2, 0, 3] = Delta**2 * rijkl[0, 0, 1, 2] / (2 * M * a * r)
    rijkl[2, 1, 0, 3] = -rijkl[1, 2, 0, 3] / Delta

    rijkl[1, 3, 2, 3] = -(r**2 + a**2) * np.sin(theta) ** 2 * rijkl[1, 0, 0, 2]
    rijkl[2, 3, 1, 3] = rijkl[1, 3, 2, 3] / Delta

    rijkl[1, 2, 1, 2] = -M * r * X / Sigma**2
    rijkl[2, 1, 1, 2] = -rijkl[1, 2, 1, 2] / Delta

    rijkl[0, 1, 2, 3] = (
        -M
        * a
        * Y
        * (2 * (r**2 + a**2) ** 2 + Delta * a**2 * np.sin(theta) ** 2)
        * np.sin(theta)
        * np.cos(theta)
        / (Delta * Sigma**3)
    )
    rijkl[0, 2, 1, 3] = (
        -M
        * a
        * Y
        * ((r**2 + a**2) ** 2 + 2 * Delta * a**2 * np.sin(theta) ** 2)
        * np.sin(theta)
        * np.cos(theta)
        / (Delta * Sigma**3)
    )

    rijkl[3, 1, 0, 2] = (
        -M
        * a
        * Y
        * (Delta + 2 * a**2 * np.sin(theta) ** 2)
        / (np.tan(theta) * Delta * Sigma**3)
    )
    rijkl[3, 2, 0, 1] = (
        -M
        * a
        * Y
        * (2 * Delta + a**2 * np.sin(theta) ** 2)
        / (np.tan(theta) * Delta * Sigma**3)
    )

    rijkl[1, 0, 0, 1] = (
        M * r * X * (2 * Delta + a**2 * np.sin(theta) ** 2) / Sigma**4
    )

    rijkl[2, 0, 0, 2] = -(
        M * r * X * (Delta + 2 * a**2 * np.sin(theta) ** 2) / Sigma**4
    )

    rijkl[1, 3, 1, 3] = (
        -M
        * r
        * X
        * ((r**2 + a**2) ** 2 + 2 * Delta * a**2 * np.sin(theta) ** 2)
        * np.sin(theta) ** 2
        / Sigma**4
    )
    rijkl[2, 3, 2, 3] = (
        M
        * r
        * X
        * (2*(r**2 + a**2) ** 2 +  Delta * a**2 * np.sin(theta) ** 2)
        * np.sin(theta) ** 2
        / Sigma**4
    )

    if config=='ulll':
        return antisymmetrize(rijkl)
    else:
        return np.einsum("ij,jklm->iklm",metric_tensor(a,r,theta),rijkl)

In [9]:
# Metric or Black Hole parameters - Mass, M and Spin Parameter, a
M = 1

In [10]:
Vin = 0.5  # Relative velocity in the units of c

alpha_s = 0.0
beta_s = np.pi / 2  # Angular Coordinates for spin Orientation

alpha_v = 0.0
beta_v = np.pi / 2  # Angular Coordinates for Velocity Orientation

sin = 0.02  # Initial Magnitude of Spin

a0 = 1 / np.sqrt(2)
r0 = 10**6
theta0 = np.pi / 2  # Parameters for Black Hole and the Particle Position


In [11]:
# Definitions
Delta = a0**2 - 2 * M * r0 + r0**2
scA = (r0**2 + a0**2) ** 2 - Delta * a0**2 * np.sin(theta0) ** 2
omega_k = 2 * M * a0 * r0 / scA
Sigma = r0**2 + a0**2 * np.cos(theta0) ** 2

In [12]:
# Initial Velocity
uhoo = np.sqrt(scA / (Delta * Sigma)) * np.array([1, 0, 0, omega_k])

In [13]:
uhoo

array([1.00000100e+00, 0.00000000e+00, 0.00000000e+00, 1.41421498e-18])

In [14]:
gk=metric_tensor(a0,r0,np.pi/2)

In [15]:
gk

array([[-9.99998000e-01,  0.00000000e+00,  0.00000000e+00,
        -1.41421356e-06],
       [ 0.00000000e+00,  1.00000200e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+12,
         0.00000000e+00],
       [-1.41421356e-06,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+12]])

In [16]:
# Tensor components calculation

v4,s4= smp.symbols('v4 s4')

V_sigma = np.array(
    [
        v4,
        Vin * np.cos(alpha_v),
        Vin * np.sin(alpha_v) * np.cos(beta_v),
        Vin * np.sin(alpha_v) * np.sin(beta_v),
    ]
)


S_sigma = np.array(
    [
        s4,
        sin * np.cos(alpha_s),
        sin * np.sin(alpha_s) * np.cos(beta_s),
        sin * np.sin(alpha_s) * np.sin(beta_s),
    ]
)

In [17]:
dum_1=np.einsum('ij,j->i',gk,uhoo)

In [18]:
v4=smp.solve(np.einsum("i,i", dum_1, V_sigma), v4)[0]
s4=smp.solve(np.einsum("i,i", dum_1, S_sigma), s4)[0]

In [19]:
# Tensor components calculation

V_sigmain = np.array(
    [
        v4,
        Vin * np.cos(alpha_v),
        Vin * np.sin(alpha_v) * np.cos(beta_v),
        Vin * np.sin(alpha_v) * np.sin(beta_v),
    ]
)


S_sigmain = np.array(
    [
        s4,
        sin * np.cos(alpha_s),
        sin * np.sin(alpha_s) * np.cos(beta_s),
        sin * np.sin(alpha_s) * np.sin(beta_s),
    ]
)

In [20]:
#Sanity Check

In [21]:
np.einsum("i,i", np.einsum("ij,j", gk, V_sigmain), uhoo), np.einsum(
    "i,i", np.einsum("ij,j", gk, S_sigmain), uhoo
)

(0, 0)

In [22]:
Gamma=1/np.sqrt(1-Vin**2)

In [23]:
Umu = (uhoo + V_sigmain) * Gamma

In [24]:
dMult=np.einsum('i,jk,j,k->i',uhoo,gk,V_sigmain,S_sigmain)

In [25]:
scS=Gamma*(dMult+S_sigmain)

In [26]:
epsilon=levi_cevita_tensor(4)

In [27]:
Sepsilon = np.einsum("mnjl,ij,kl->mnik",epsilon, gk, gk)

In [28]:
scM=1e-6

In [29]:
SinM=scS/scM

In [30]:
MSinM=sin/scM

In [31]:
Suvin=np.einsum('ijkl,k,l->ij',Sepsilon,Umu,SinM)

In [32]:
Suvin

array([[0, 0, 0, 0],
       [0, 0, 3.46944695195361e-18, 0],
       [0, -3.46944695195361e-18, 0, -20000.0066666500],
       [0, 0, 20000.0066666500, 0]], dtype=object)

In [33]:
SRijkl=kerr_riemann_tensor(a0,r0,np.pi/2,1,'llll')